In [ ]:
from acdh_tei_pyutils.tei import TeiReader
from acdh_tei_pyutils.utils import get_xmlid
from tqdm import tqdm
from icecream import ic
from normdata.utils import import_from_normdata

In [ ]:
source_file = "https://raw.githubusercontent.com/hermann-bahr/bahr-index/main/tsn/listwork_TSN_1.xml"

In [ ]:
doc = TeiReader(source_file)

In [ ]:
nsmap = doc.nsmap
bahr_url = "https://hermanbahrtextverzeichnis/"
bahr_domain = "hermanbahrtextverzeichnis"
bahr_col, _ = Collection.objects.get_or_create(name="Bahr Textverzeichnis")

In [ ]:
for x in tqdm(doc.any_xpath(".//tei:bibl[@xml:id]")):
    entity = False
    try:
        hbtv_uri = get_xmlid(x)
    except KeyError:
        continue
    hbtv_url = f"{bahr_url}{hbtv_uri}"
    uri, _ = Uri.objects.get_or_create(uri=hbtv_url, domain=bahr_domain)
    try:
        name = x.xpath("./tei:title[1]/text()", namespaces=nsmap)[0]    
    except IndexError:
        print(hbtv_url)
        continue
    try:
        entity = uri.entity.get_child_entity()
    except:
        entity = Work.objects.create(name=name)
        entity.collection.add(bahr_col)
        uri.entity = entity
        uri.save()